In [19]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tkinter import Tk, Button, Label
from PIL import Image as PILImage, ImageTk

# Step 1: Define your model architecture with more layers and dropout to prevent overfitting
def create_model():
    model = Sequential()
    
    # First Conv Layer
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second Conv Layer
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third Conv Layer
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten the output of the final Conv layer
    model.add(Flatten())
    
    # Fully connected layers with dropout for regularization
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Dropout to prevent overfitting
    model.add(Dense(7, activation='softmax'))  # Output layer for 7 emotions
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Step 2: Data Augmentation for improved generalization
train_data_dir = r'C:\Users\anshu\Emoji-Generator\data\train'  # Update with your data path
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

# Step 3: Create and train the model
emotion_model = create_model()

# Learning Rate Scheduler to reduce the learning rate as training progresses
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the model
emotion_model.fit(train_generator, epochs=100, callbacks=[reduce_lr])  # Adjust the number of epochs as needed

# Save the model weights
emotion_model.save_weights('emotion_model_final.weights.h5')

# Step 4: Load the model weights (for later use)
def load_model():
    model = create_model()
    model.load_weights('emotion_model_final.weights.h5')
    return model

# Step 5: Capture an image from the webcam and predict the emotion
def capture_and_predict(model):
    emoji_dict = {
        0: cv2.imread(r'C:\Users\anshu\Emoji-Generator\emojis\Angry.png', -1),
        1: cv2.imread(r'C:\Users\anshu\Emoji-Generator\emojis\disgusted.png', -1),
        2: cv2.imread(r'C:\Users\anshu\Emoji-Generator\emojis\happy.png', -1),
        3: cv2.imread(r'C:\Users\anshu\Emoji-Generator\emojis\sad.png', -1),
        4: cv2.imread(r'C:\Users\anshu\Emoji-Generator\emojis\surprised.png', -1),
        5: cv2.imread(r'C:\Users\anshu\Emoji-Generator\emojis\neutral.png', -1),
        6: cv2.imread(r'C:\Users\anshu\Emoji-Generator\emojis\fearful.png', -1)
    }


Found 27544 images belonging to 7 classes.
Epoch 1/100
861/861 ━━━━━━━━━━━━━━━━━━━━ 78s 85ms/step - accuracy: 0.2338 - loss: 1.8295 - learning_rate: 1.0000e-04
Epoch 2/100
861/861 ━━━━━━━━━━━━━━━━━━━━ 68s 78ms/step - accuracy: 0.2608 - loss: 1.7997 - learning_rate: 1.0000e-04
Epoch 3/100
861/861 ━━━━━━━━━━━━━━━━━━━━ 49s 56ms/step - accuracy: 0.2664 - loss: 1.7822 - learning_rate: 1.0000e-04
Epoch 4/100
861/861 ━━━━━━━━━━━━━━━━━━━━ 45s 52ms/step - accuracy: 0.2649 - loss: 1.7818 - learning_rate: 1.0000e-04
Epoch 5/100
861/861 ━━━━━━━━━━━━━━━━━━━━ 63s 73ms/step - accuracy: 0.2706 - loss: 1.7769 - learning_rate: 1.0000e-04
Epoch 6/100
861/861 ━━━━━━━━━━━━━━━━━━━━ 46s 53ms/step - accuracy: 0.2708 - loss: 1.7687 - learning_rate: 1.0000e-04
Epoch 7/100
861/861 ━━━━━━━━━━━━━━━━━━━━ 124s 143ms/step - accuracy: 0.2758 - loss: 1.7611 - learning_rate: 1.0000e-04
Epoch 8/100
861/861 ━━━━━━━━━━━━━━━━━━━━ 37s 43ms/step - accuracy: 0.2800 - loss: 1.7559 - learning_rate: 1.0000e-04
Epoch 9/100
861/861


KeyboardInterrupt



In [18]:
import cv2
import numpy as np

# Function to display the image and emoji side by side
def display_image_with_emoji(captured_image, emoji):
    # Convert the emoji from RGBA (with alpha) to RGB by removing the alpha channel
    if emoji.shape[2] == 4:
        emoji = cv2.cvtColor(emoji, cv2.COLOR_BGRA2BGR)

    # Resize emoji to match the captured image height
    emoji_resized = cv2.resize(emoji, (captured_image.shape[1], captured_image.shape[0]))
    
    # Combine the two images horizontally
    combined_image = np.hstack((captured_image, emoji_resized))
    
    # Show the combined image
    cv2.imshow('Captured Image and Emoji', combined_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Open webcam to capture an image
cap = cv2.VideoCapture(0)
print("Press 'c' to capture an image")

while True:
    ret, frame = cap.read()
    cv2.imshow('Webcam', frame)
    
    key = cv2.waitKey(1)
    if key & 0xFF == ord('c'):
        # Capture the image
        cv2.imwrite('captured_image.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()

# Process the image for prediction
img = cv2.imread('captured_image.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (48, 48))
img = img.astype('float32') / 255.0
img = np.reshape(img, (1, 48, 48, 1))

# Predict the emotion using the loaded model
prediction = loaded_model.predict(img)
emotion_index = np.argmax(prediction)
print(f'Predicted Emotion Index: {emotion_index}')

# Display the emoji corresponding to the predicted emotion
emoji = emoji_dict[emotion_index]

# Display the captured image and emoji side by side
captured_image = cv2.imread('captured_image.jpg')  # Load the original colored captured image
display_image_with_emoji(captured_image, emoji)

# Main flow
if __name__ == "__main__":
    # Load the model (this will load the weights saved earlier)
    loaded_model = load_model()
    # Capture an image and predict the emotion
    capture_and_predict(loaded_model)


Press 'c' to capture an image
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted Emotion Index: 6
